In [33]:
import pandas as pd
import numpy as np

import datetime

from tqdm._tqdm_notebook import tqdm_notebook

tqdm_notebook.pandas()

import xgboost as xgb 

from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_squared_error

import graphviz

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def fillnan(data, years_c):
    for name in years_c:
        #заменяем троеточия
        data[name] = data[name].apply(lambda x: np.nan if x==('..') else float(x))

    return data

def dropempt(data, years_c):
    data = fillnan(data, years_c)
    return data.dropna(thresh=10)

def Clearing(data):
    years_c = [item for ind, item in enumerate(np.array(data.columns)) if item not in ['Country Name', 'Country Code','Series Code','Series Name']]
    print ('Before clear:', len(data))

    data = dropempt(data, years_c)

    print ('After clear:', len(data))
    return data

In [3]:
def Make_region(code, reg='region', by = 'name'):
    #print code
    if code in data_cnt[by].values:
        #print code
        name = (data_cnt[by][data_cnt[by] == code].index.tolist())[0]
        return data_cnt[reg].at[name]
    else:
        return np.nan

def Sorting(data, reg='region'):
    #if data['Country Name'].get_value(len(data)-5) == np.nan:
    #    data=data.drop(data.index[[range(len(data)-5, len(data))]]).reset_index()
    if 'Region' not in data.columns:
        data['Region'] = data['Country Name'].progress_apply(lambda x: Make_region(x, reg))
        data['Region'].dropna()
        return data.reset_index()
    else:
        return data

In [4]:
df = pd.read_excel('data/Data_Extract_From_Gender_Statistics.xlsx', encoding = 'utf8').\
            append(pd.read_excel('data/Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', encoding='utf8')).\
            append(pd.read_excel('data/Data_Extract_From_Millennium_Development_Goals.xlsx', encoding='utf8')).\
            append(pd.read_excel('data/Data_Extract_From_Health_Nutrition_and_Population_Statistics_by_Wealth_Quintile.xlsx',encoding='utf8'))

data_cnt = pd.read_csv('all.csv', encoding='utf8')
df = Clearing(df)
defen = pd.read_excel('data/Data_Extract_From_Gender_Statistics.xlsx', sheet_name=1, encoding = 'utf8').\
               append(pd.read_excel('data/Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', sheet_name=1, encoding='utf8')).\
               append(pd.read_excel('data/Data_Extract_From_Millennium_Development_Goals.xlsx', sheet_name=1, encoding='utf8'))

df = Sorting(df, 'sub-region')
df.drop_duplicates(inplace = True)
df.drop(columns = 'index', inplace=True)

Before clear: 398541
After clear: 104445


HBox(children=(IntProgress(value=0, max=104445), HTML(value='')))

In [27]:
def makexy(df, country = 'RUS'):
    #Years
    years_c = [item for ind, item in enumerate(np.array(df.columns)) if item not in ['Region','Country Name', 'Country Code','Series Code','Series Name', 'index']]
    
    #split country
    df = df[(df['Country Code']== country)].copy()
    df.drop_duplicates(inplace = True)
    
    y = df[df['Series Name'] == 'Life expectancy at birth, total (years)'][years_c].interpolate(axis=1).T
    
    if y.shape[1] == 1:
        y.columns = ['Life expectancy at birth, total (years)']
    else:
        return pd.DataFrame(), pd.DataFrame()
    
    #Transponenting and then adding 
    mask = ((df['Series Name'] != 'Life expectancy at birth, male (years)') & 
            (df['Series Name'] != 'Life expectancy at birth, female (years)') &
            (df['Series Name'] != 'Life expectancy at birth, total (years)'))
    
    X = df[mask][years_c].T
    X.columns = df[(df['Country Code']== country) & mask]['Series Name']
    X.reset_index(inplace = True, drop = True)
    del df
    
    X['Years'] = [int(name.split()[0]) for name in years_c]
    X['Region_code'] = int(Make_region(country,'region-code', by = 'alpha-3'))
    X['Reion_sub_code'] = int(Make_region(country, 'sub-region-code', 'alpha-3'))
    X['Country_code'] = int(Make_region(country, 'country-code', 'alpha-3'))
    return X.reset_index(drop = True), y.reset_index(drop = True)

def train_set(df, region = 'Eastern Europe'):
    x_train, y_train = pd.DataFrame(), pd.DataFrame()
    for country in df[df['Region'] == region]['Country Code'].unique():
        tmp = makexy(df, country)
        x_train = pd.concat([tmp[0], x_train], axis=0).fillna(-999)
        y_train = pd.concat([tmp[1], y_train], axis=0).fillna(-999)
    return x_train.as_matrix(), y_train.as_matrix(), x_train.columns
    

In [42]:
def Testing_grid_xgb(X_train, Y_train):
    cv = TimeSeriesSplit(n_splits=4)
    clf = xgb.XGBRegressor()        
    parameters = {
        'scale_pos_weight' : [1, 1.5],
        'objective':['reg:linear'],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [3 ,6],
        'min_child_weight': [3 ,4, 5],
        'max_delta_step': [0, 0.2, 0.8],
        'reg_lambda': [0.5, 0.8, 0],
        'reg_alpha': [0.5, 0.8, 0],
        'silent': [1],
        'subsample': [0.2, 0.05],
        'colsample_bytree': [0.7, 0.8, 1],
        'n_estimators': [100 ,250, 450],
        'base_score' : [0.5, 1],
        'gamma': [0, 0.5, 1]
    }
    
    start_time = datetime.datetime.now()
    gs = RandomizedSearchCV(clf, scoring='neg_mean_squared_error', param_distributions=parameters, 
                      cv=cv, n_jobs = -1, verbose=True,  n_iter=500)
    gs.fit(X_train,Y_train)
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    
    means = gs.cv_results_['mean_test_score']
    
    print (max(means))
    return gs.best_params_

In [43]:
def Resmaking(df):
    for region in tqdm_notebook(df['Region'].dropna().unique()):
        tmp = train_set(df, region)
        X_train, X_test, y_train, y_test = train_test_split(tmp[0], tmp[1], test_size=0.25, random_state=42)
        p = Testing_grid_xgb(X_train, y_train)
        
        model = xgb.XGBRegressor(max_depth=p['max_depth'], learning_rate = p['learning_rate'], 
                                 n_estimators = p['n_estimators'], objective = p['objective'], 
                                 gamma = p['gamma'], min_child_weight = p['min_child_weight'], 
                                 max_delta_step = p['max_delta_step'], subsample = p['subsample'], 
                                 colsample_bytree = p['colsample_bytree'], reg_alpha = p['reg_alpha'], 
                                 reg_lambda = p['reg_lambda'], scale_pos_weight = p['scale_pos_weight'], 
                                 base_score = p['base_score'])
        model.fit(X_train, y_train)
        
        print ('MSE : {0:f}%, \nIn {1:s}'.format(100-mean_squared_error(model.predict(X_test), y_test), region))
        
        model.fit(tmp[0], tmp[1])
        
        rez = pd.DataFrame(model.feature_importances_, index = tmp[2]).sort_values(by = [0], ascending=False)
        rez[rez[0]!=0].to_csv('Importance\Varible_in_'+str(region)+'.csv')

In [44]:
Resmaking(df)

HBox(children=(IntProgress(value=0, max=22), HTML(value='')))

Fitting 4 folds for each of 500 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 1024 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:   19.0s finished


Time elapsed: 0:00:19.511576
-3.622864795057584
MSE : 99.586575%, 
In Southern Asia
Fitting 4 folds for each of 500 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 936 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:   13.5s finished


...

Time elapsed: 0:00:10.409158
-4.062276508074423
MSE : 95.622427%, 
In Northern America
Fitting 4 folds for each of 500 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 936 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    9.5s finished


Time elapsed: 0:00:09.776950
-0.9447898151738808
MSE : 99.877525%, 
In Polynesia
Fitting 4 folds for each of 500 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    8.3s finished


Time elapsed: 0:00:08.681005
-5.2554911972226535
MSE : 99.758044%, 
In Micronesia



In [41]:
RandomizedSearchCV?